### request
- 1) json -- if there is api
- 2) html -(parsing by using css selector for crawling)- bs4(string data)
- 3) selenium -- css selector (by using css selector and xpath)
- 4) scrapy - package for crawling, use css selector, xpath 
- number of each row is about priority in using method.

In [1]:
import requests
# 서버에 request 요청을 하고 response 응답을 받는 패키지 
import time

In [2]:
from bs4 import BeautifulSoup 

In [3]:
# dark sky api 날씨 정보를 가지고 오는 패키지 
import forecastio

ModuleNotFoundError: No module named 'forecastio'

#### 네이버 주식 데이터 크롤링
- api 사용 : json 파싱해서 데이터를 크롤링합니다.
- 네이버 주식 페이지에서 주식 데이터를 가져와서 판다스 데이터프레임으로 만듭니다.

### request
- 1) json -- if there is api
- 2) html -(parsing by using css selector for crawling)- bs4(string data)
- 3) selenium -- css selector (by using css selector and xpath)
- 4) scrapy - package for crawling, use css selector, xpath 
- number of each row is about priority in using method.

##### 네이버 주식 데이터 가져오기
- api 사용 : json 파싱을 합니다.
- 네이버 주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만듭니다.
- 웹서버보다 모바일 페이지가 크롤링이 더 쉬운 경우가 있으므로 염두에 둡니다.

url 분석

In [5]:
url = "http://m.stock.naver.com/api/json/sise/\
siseListJson.nhn?menu=market_sum&sosok={0}&pageSize={0}&page={}"

#### 분석 결과
- sosok = 보여지는 주식 카테고리(코스피,코스닥...)
- pageSize = 한번에 보여지는 주식 데이터의 갯수
- page = 더보기를 통해 보여지는 주식이 달라지지는데 그 순서를 의미합니다.

- result: {
    - totCnt: 1274,
    - ms: "OPEN",
    - itemList: [
- result 보다 낮은 level로 factor들이 들어있는 data format을 가지고 있습니다.

make_url 함수 만들기 -- 인자에 따라서 보여지는 url의 json data가 바뀝니다.

In [6]:
http://m.stock.naver.com/api/json/sise/siseListJson.nhn?m\
    enu=market_sum&sosok=1&pageSize=5&page=5

SyntaxError: invalid syntax (<ipython-input-6-e62645cab4d4>, line 1)

In [7]:
def make_url(pageSize=10,page=1):
    return 'http://m.stock.naver.com/api/json/sise\
/siseListJson.nhn?menu=market_sum&sosok=0&pageSize='+str(pageSize)+'&page='+str(page)
#default argument를 정해주고 후에 필요시, 변경해주도록 합니다.

In [8]:
make_url()

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0&pageSize=10&page=1'

In [9]:
def get_data(url):
    response = requests.get(url) # url 을 요청합니다.
    json_info = response.json() # json data format으로 parsing
    companys = json_info['result']['itemList'] #우리가 필요한 데이터는 itemList 배열입니다.
    df = pd.DataFrame(columns = ['종목','시세','전열비','등락율','시가총액','거래량'])
    #df이라는 dataframe 변수를 생성해주고 column을 할당해줍니다. 여기서 columns의 갯수는
    #데이터의 갯수와 같게 설정해주어야 합니다.
    for company in companys:
        df.loc[len(df)] = { #위에서 부터 하나씩 아래로 데이터들을 for 반복문을 통해 채워나갑니다.
            '종목':company['nm'],
            '시세':company['nv'],
            '전열비':company['cv'],
            '등락율':company['cr'],
            '시가총액':company['mks'],
            '거래량':company['aq'],
        }#dictionary datatype으로 dataframe의 value들을 채워넣어가 줍니다.
    return df

In [10]:
# resonse.text -- html code
# json _info -- json data parsed

In [11]:
url = make_url(100,2)
df = get_data(url)
print(len(df))
df.tail()

100


,종목,시세,전열비,등락율,시가총액,거래량
95,F&F,68400,1900,2.86,10534,57899
96,삼양홀딩스,121000,3000,2.54,10363,22407
97,LG상사,26450,50,0.19,10252,245911
98,미래에셋생명,5790,-200,-3.34,10249,576843
99,종근당,103500,0,0.00,10225,30785


Dart Sky API

In [12]:
secret_key = 'c411a7d620867c8a2eb8e81fc5889502'

In [13]:
def forecast(lat,lng):
    url =  'https://api.darksky.net/forecast/{0}/{1},{2}'.format(secret_key,lat,lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["hourly"]["summary"], json_info["timezone"]

In [14]:
lat = 37.5124413
lng = 126.9540519
forecast(lat,lng)

('Mostly cloudy throughout the day.', 'Asia/Seoul')

In [15]:
#forecastio
def forecast2(lat,lng):
    forecast = forecastio.load_forecast(secret_key,lat,lng)
    byHour = forecast.hourly()
    return byHour,summary

In [16]:
lat = 37.5124413
lng = 126.9540519
forecast(lat,lng)

('Mostly cloudy throughout the day.', 'Asia/Seoul')

##### beautifulsoup을 이용하는 방법
- 문자열로 되어있는 html 코드를 css selector나 xpath를 이용해서 특정 엘리먼트의 데이터를 쉽게 가져올 수 있도록 하는 패키지

In [4]:
### 네이버 검색어 순위 데이터를 크롤링 하겠습니다.
def naver_top20():
    df = pd.DataFrame(columns = ['rank','keyword'])
    response = requests.get('http://naver.com')
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords= dom.select(".ah_roll .ah_l .ah_item")
    for keyword in keywords:
        df.loc[len(df)] = {
            'rank' : keyword.select_one('.ah_r').text,
            'keyword' : keyword.select_one('.ah_k').text,
        }
    return df

[<li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">1</span>
 <span class="ah_k">김종필</span>
 </a>
 </li>, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">2</span>
 <span class="ah_k">김영삼</span>
 </a>
 </li>, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">3</span>
 <span class="ah_k">쌈디</span>
 </a>
 </li>, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">4</span>
 <span class="ah_k">노태우</span>
 </a>
 </li>, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">5</span>
 <span class="ah_k">독일 스웨덴</span>
 </a>
 </li>, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">6</span>
 <span class="ah_k">벨기에 튀니지</span>
 </a>
 </li>, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">7</span>
 <span class="ah_k">백종원

In [18]:
naver_df = naver_top20()
naver_df

,rank,keyword
0,1,이집트 우루과이
1,2,모로코 이란
2,3,포항 약국
3,4,러시아 사우디 하이라이트
4,5,쌈디
5,6,블랙핑크
6,7,포르투갈 스페인
7,8,살라
8,9,안희정
9,10,국가장학금


In [19]:
import time
ls = []
for idx in range(3):
    print(idx)
    df = naver_top20()
    ls.append(df)
    time.sleep(10)

0
1
2


In [20]:
def daum_top10():
    df = pd.DataFrame(columns = ['rank','keyword'])
    response = requests.get('http://daum.net')
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords= dom.select(".realtime_part > ol.list_hotissue.issue_row > li")
    for keyword in keywords:
        df.loc[len(df)] = {
            'rank' : keyword.select_one('.ir_wa').text,
            'keyword' : keyword.select_one('.link_issue').text,
        }
    return df

In [21]:
daum_df = daum_top10()
daum_df

,rank,keyword
0,1위,심영은
1,2위,사이먼도미닉
2,3위,블랙핑크
3,4위,김무성
4,5위,국가장학금
5,6위,박원빈
6,7위,안희정
7,8위,김성태
8,9위,이집트 우루과이
9,10위,안신애


In [24]:
ls3=[]
for i in range(len(ls1)):
    for j in range(len(ls2)):
        if ls1[i]==ls2[j]:
            ls3.append(ls1[i])
ls3

['국가장학금', '블랙핑크', '심영은', '안희정', '이집트 우루과이']

In [25]:
ls3 = [ls1[i] for i in range(len(ls1)) for j in range(len(ls2)) if ls1 == ls2[j]]
ls3

[]

In [1]:
(set(ls2) & (set(ls3))

SyntaxError: unexpected EOF while parsing (<ipython-input-1-e36264436f1f>, line 1)